<a href="https://colab.research.google.com/github/adamiaonr/cveml/blob/main/projects/scripts/notebooks/project_image_data_augmentation_adamiaonr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Image Data Augmentation

[![Open In Colab <](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ShawnHymel/computer-vision-with-embedded-machine-learning/blob/master/2.3.5%20-%20Project%20-%20Data%20Augmentation/project_image_data_augmentation.ipynb)

This is an example for creating an augmented dataset. It will transform input images to create a series of augmented samples that are saved in a new output directory.

Create a folder named "dataset" in the /content directory and upload your images there. The images should be divided into their respective classes, where each class has its own folder with the name of the class. For example:

<pre>
/content
    |- dataset
        |- background
        |- capacitor
        |- diode
        |- led
        |- resistor
</pre>

The original images along with their transforms will be saved in the output directory. Each output file will be the original filename appended with "_{num}" where {num} is some incrementing value based on the total number of transforms performed per image.

For example, if you have a file named "0.png" in /content/dataset/resistor, it will become "0_0.png" in /content/output/resistor. The first transform will be "0_1.png", the second transform will be "0_2.png" and so on.

Run each of the cells paying attention to their contents and output. Fill out the necessary parts of the functions where you find the following comment:

```
# >>> ENTER YOUR CODE HERE <<<
```

Author: EdgeImpulse, Inc.<br>
Date: August 3, 2021<br>
Modified by: adamiaonr@gmail.com<br>
Last modified: January 2, 2022<br>
License: [Apache-2.0](apache.org/licenses/LICENSE-2.0)<br>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import PIL

import skimage.transform
import skimage.util

In [ ]:
### Settings

# Location of dataset and output folder
DATASET_PATH = "/content/dataset"
OUT_PATH = "/content/output"
OUT_ZIP = "augmented_dataset_v2_0_6.zip"

# File format to use for new dataset
IMG_EXT = ".png"

# You are welcome to change the seed to get different augmentation effects
SEED = 42
random.seed(SEED)


In [ ]:
### Create output directory
!rm -rf "{OUT_PATH}"
!rm -rf "{OUT_ZIP}"

try:
  os.makedirs(OUT_PATH)
except FileExistsError:
  print("WARNING: Output directory already exists. Check to make sure it is empty.")


## Transform Functions

Create one or more functions that transform an input image.

In [ ]:
### Example: Function to create 3 new flipped images of the input
def create_flipped(img):

  # Create a list of flipped images
  flipped = []
  flipped.append(np.fliplr(img))
  flipped.append(np.flipud(img))
  flipped.append(np.flipud(np.fliplr(img)))

  return flipped


In [ ]:
# >>> ENTER YOUR CODE HERE <<<
# Create one or more functions that create transforms of your images

# rotation : rotates image with the provided angles ([-135, -45, 45, 135] by default)
def create_rotated(img, angles = [-135, -45, 45, 135], flip = False):
  rotated = []
  for angle in angles:
    img_rotated = skimage.transform.rotate(img, angle = angle, mode='edge', preserve_range=True).astype(np.uint8)
    rotated.append(img_rotated)

    if flip:
      rotated.append(np.fliplr(img_rotated))

  return rotated

# translation : creates a list of translations of a source image along {x, y} axis, in the form {x * (1 + a), y * (1 + b)}
# the (a, b) coefficients to apply are passed as a list.
def create_translated(img, translation_coef = [0.0, .25, .5, .75], flip = False):
  translated = []

  height = img.shape[0]
  width = img.shape[1]

  for c in translation_coef:
    tr_x = [ round(( random.random() * k ) * ( width / 2 ))  for k in [ c, c, -c, -c ] ]
    for c in translation_coef:
      tr_y = [ round(( random.random() * k ) * ( height / 2 )) for k in [ c, c, -c, -c ] ]
      for i, p in enumerate([(x, y) for x, y in zip(tr_x, tr_y)]):
        translation = skimage.transform.AffineTransform(translation=(p[0], p[1]))
        img_trnsltd = skimage.transform.warp(img, translation, mode='edge', preserve_range=True).astype(np.uint8)
        translated.append(img_trnsltd)
        
        if flip:
          translated.append(np.flipud(img_trnsltd))

  return translated

# scale & crop
def create_scaled(img):
  
  scaled = []

  # choose a 'soft' scaling factor 
  scale_factor = 1.3

  # scale image
  img_scaled = skimage.transform.rescale(
      img, 
      scale = scale_factor, 
      anti_aliasing = True, 
      multichannel = True,
      preserve_range = True).astype(np.uint8)
  # print("scale change (%d, %d) : (%d, %d)" % (img.shape[1], img.shape[0], img_scaled.shape[1], img_scaled.shape[0]))

  # crop at the center and append to output list
  crop_x = int(np.floor((img_scaled.shape[0] - img.shape[0]) / 2))
  crop_y = int(np.floor((img_scaled.shape[1] - img.shape[1]) / 2))
  # print("cropping point : (%d, %d)" % (crop_x, crop_y))

  scaled.append(img_scaled[crop_y:(crop_y + img.shape[1]), crop_x:(crop_x + img.shape[0]), :])
  scaled.append(np.fliplr(img_scaled[crop_y:(crop_y + img.shape[1]), crop_x:(crop_x + img.shape[0]), :]))
  # print("scaled & cropped : (%d, %d)" % (img_scaled_cropped.shape[0], img_scaled_cropped.shape[1]))

  # create scaled & cropped at {flipped, non-flipped} * {N, S, W, E, NE, SE, SW, NW}
  crop_x_list = [ round(( random.random() * k + 1 ) * crop_x ) for k in [ 1.0, 1.0, -1.0, -1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0 ] ]
  crop_y_list = [ round(( random.random() * k + 1 ) * crop_y ) for k in [ 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0 ] ]

  for i, crop in enumerate([(x, y) for x, y in zip(crop_x_list, crop_y_list)]):
    if (i % 2):
      scaled.append(np.fliplr(img_scaled[crop[1]:(crop[1] + img.shape[1]), crop[0]:(crop[0] + img.shape[0]), :]))
    else:
      scaled.append(img_scaled[crop[1]:(crop[1] + img.shape[1]), crop[0]:(crop[0] + img.shape[0]), :])

  return scaled


In [ ]:
# testing area

img = PIL.Image.open(os.path.join(DATASET_PATH, 'resistor/001.png'))
img = np.asarray(img)

# create simple array of translated images
translated = create_translated(img, translation_coef = [1.0], flip = False)
figs, axs = plt.subplots(1, len(translated), figsize=(30, 30))

for i in range(len(translated)):
  axs[i].imshow(translated[i], vmin=0, vmax=255)

plt.show()

# create array of rotated images
rotated = create_rotated(img, angles = [-45, -22.5, -11.25, 11.25, 22.5, 45])
figs, axs = plt.subplots(1, len(rotated) + 1, figsize=(30, 30))
axs[0].imshow(img, vmin=0, vmax=255)
for i in range(len(rotated)):
  axs[i + 1].imshow(rotated[i], vmin=0, vmax=255)

plt.show()

# create array of rotations, applied over translations
for img_trnslt in create_translated(img, translation_coef = [0.0, .25, .5]):
  trans_rotated = create_rotated(img_trnslt, angles = [-11.25, 11.25])
  figs, axs = plt.subplots(1, len(trans_rotated) + 1, figsize=(30, 30))
  axs[0].imshow(img, vmin=0, vmax=255)
  for i in range(len(trans_rotated)):
    axs[i + 1].imshow(trans_rotated[i], vmin=0, vmax=255)

  plt.show()

# scaled = create_scaled(img)
# figs, axs = plt.subplots(1, len(scaled), figsize=(30, 30))
# for i in range(len(scaled)):
#   axs[i].imshow(scaled[i], vmin=0, vmax=255)

# plt.show()


## Perform Transforms

Call your functions to create a set of augmented data.

In [ ]:
### Function to open image and create a list of new transforms
# NOTE: You will need to call your functions here!
def create_transforms(file_path):

  # Open the image
  img = PIL.Image.open(file_path)

  # Convert the image to a Numpy array (keep all color channels)
  img_array = np.asarray(img)

  # Add original image to front of list
  img_tfs = []
  img_tfs.append([img_array])

  # Perform transforms (call your functions)
  # img_tfs.append(create_flipped(img_array))
  # >>> ENTER YOUR CODE HERE <<<
  # e.g. img_tfs.append(create_translations(img_array, 2))

  # add translations with coefficients [.0, .25, .5, .75]
  img_tfs.append(create_translated(img_array, translation_coef = [.0, .25, .5, .75], flip = True))
  # add rotations with angles [-22.5, -11.25, 11.25, 22.5] and translation coefficients [.0, .25]
  # for img_trnslt in create_translated(img_array, translation_coef = [.0, .25]):
  #   img_tfs.append(create_rotated(img_trnslt, angles = [-22.5, -11.25, 11.25, 22.5], flip = True))

  # Flatten list of lists (to create one long list of images)
  img_tfs = [img for img_list in img_tfs for img in img_list]

  return img_tfs


In [ ]:
### Load all images, create transforms, and save in output directory

# Find the directories in the dataset folder (skip the Jupyter Notebook checkpoints hidden folder)
for label in os.listdir(DATASET_PATH):
  class_dir = os.path.join(DATASET_PATH, label)
  if os.path.isdir(class_dir) and label != ".ipynb_checkpoints":

    # Create output directory
    out_path = os.path.join(OUT_PATH, label)
    os.makedirs(out_path, exist_ok=True)

    # Go through each image in the subfolder
    for i, filename in enumerate(os.listdir(class_dir)):

      # Skip the Jupyter Notebook checkpoints folder that sometimes gets added
      if filename != ".ipynb_checkpoints":

        # Get the root of the filename before the extension
        file_root = os.path.splitext(filename)[0]

        # Do all transforms for that one image
        file_path = os.path.join(DATASET_PATH, label, filename)
        img_tfs = create_transforms(file_path)

        # Save images to new files in output directory
        for i, img in enumerate(img_tfs):

          # Create a Pillow image from the Numpy array
          img_pil = PIL.Image.fromarray(img)

          # Construct filename (<orignal>_<transform_num>.<EXT>)
          out_file_path = os.path.join(out_path, file_root + "_" + str(i) + IMG_EXT)

          # Convert Numpy array to image and save as a file
          img_pil = PIL.Image.fromarray(img)
          img_pil.save(out_file_path)

In [ ]:
### Zip our new dataset (use '!' to call Linux commands)
!zip -r -q "{OUT_ZIP}" "{OUT_PATH}"
